<a href="https://colab.research.google.com/github/konderal333/HGT-2022-EmDomArDon/blob/main/bert2bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load models and data

In [ ]:
!git clone https://github.com/konderal333/HGT-2022-EmDomArDon.git

fatal: destination path 'HGT-2022-EmDomArDon' already exists and is not an empty directory.


In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
df = pd.read_csv('/content/HGT-2022-EmDomArDon/cikkek_10k_cleanedv1.csv')

In [ ]:
print(df.isnull().values.any())
print(sum(df.duplicated()))

True
87


In [ ]:
df = df.dropna()
df = df.drop_duplicates()

In [ ]:
df = df.apply(lambda x: x.str.strip())
df['Textbody'] = df['Textbody'].apply(lambda x: re.sub(r"(\.)([A-Z])", r'\1 \2', str(x))) # add missing whitespace between sentences

df['Lead'] = df['Title'] + '. ' + df['Headline'] # maybe only headline and drop title?
df = df.drop(['Title', 'Headline'],axis=1)
df.head()

,Textbody,Lead
0,Az Esemény hazai szeptember végi mozis premier...,Fájóan aktuális és elviselhetetlenül feszült a...
1,Majdnem duplájába kerülhet a Márton-napi libas...,Brutális mértékben drágul a liba- és kacsahús....
2,"Nagyon úgy tűnt, hogy a Bayern München ismét l...","Két góllal vezetett a Bayern München, a 95. pe..."
3,Korlátozást rakott a Meta Kanye West Instagram...,Antiszemita tartalom miatt korlátozták Kanye W...
4,"Elsodortak egy nézőt a hétvégi ózdi ralin, és ...",Meghalt egy néző az ózdi ralin. Elsodorta az e...


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('SZTAKI-HLT/hubert-base-cc')

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# bert2bert
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5" -O model_bert2bert.tar.gz && rm -rf /tmp/cookies.txt

# model
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GEdOmmKZEJOD2Ei28FtVEf534-nBEcBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1GEdOmmKZEJOD2Ei28FtVEf534-nBEcBd" -O model.tar.gz && rm -rf /tmp/cookies.txt

Downloading...
From: https://drive.google.com/uc?id=1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5
To: /content/model_bert2bert.tar.gz
100% 2.65G/2.65G [00:10<00:00, 246MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1GEdOmmKZEJOD2Ei28FtVEf534-nBEcBd
To: /content/model.tar.gz
100% 927M/927M [00:05<00:00, 177MB/s]


In [ ]:
import shutil
shutil.unpack_archive('model.tar.gz', 'model')
shutil.unpack_archive('model_bert2bert.tar.gz', 'bert2bert')

In [ ]:
import torch
from transformers import EncoderDecoderModel

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

bert2bert_model_path = '/content/bert2bert/model/pytorch_model.bin'
bert2bert_config = '/content/bert2bert/model/config.json'

model_path = '/content/model/model/pytorch_model.bin'
config = '/content/model/model/config.json'

bert2bert = EncoderDecoderModel.from_pretrained(bert2bert_model_path, config=bert2bert_config).to(device)
model = EncoderDecoderModel.from_pretrained(model_path, config=config).to(device)

https://colab.research.google.com/drive/1WIk2bxglElfZewOHboPFNj8H44_VAyKE?usp=sharing&fbclid=IwAR2-pdnudSutvgqIlUUg4NP1Q0wJO9NQbteg_wVL9n8gsS2yMTfMp9YfC6w#scrollTo=bYmdx-W1NAky

## Data stats

In [ ]:
# # map article and summary len to dict as well as if sample is longer than 512 tokens
# def map_to_length(x):
#   x["article_len"] = len(tokenizer(x["Textbody"]).input_ids)
#   x["article_longer_512"] = int(x["article_len"] > 512)
#   x["summary_len"] = len(tokenizer(x["Lead"]).input_ids)
#   x["summary_longer_64"] = int(x["summary_len"] > 64)
#   x["summary_longer_128"] = int(x["summary_len"] > 128)
#   return x

In [ ]:
# data_stats = df.apply(map_to_length, axis=1)

In [ ]:
# def compute_and_print_stats(x):
#   print(
#       "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
#           x["article_len"],
#           x["article_longer_512"],
#           x["summary_len"],
#           x["summary_longer_64"],
#           x["summary_longer_128"],
#       )
#   )

In [ ]:
# output = data_stats.apply(compute_and_print_stats, axis=1)

## Process data

In [ ]:
def process_data_to_model_inputs(batch):
        # Tokenize the input and target data
        inputs = tokenizer(batch['Textbody'], padding='max_length', truncation=True, max_length=512)
        outputs = tokenizer(batch['Lead'], padding='max_length', truncation=True, max_length=512)

        batch['input_ids'] = inputs.input_ids
        batch['attention_mask'] = inputs.attention_mask
        batch['decoder_input_ids'] = outputs.input_ids
        batch['decoder_attention_mask'] = outputs.attention_mask
        batch['labels'] = outputs.input_ids.copy()

        # batch['labels'] = [[-100 if token == tokenizer.pad_token_id else token for token in labels]
        #                    for labels in batch['labels']]
        batch['labels'] = [-100 if token == tokenizer.pad_token_id else token for token in batch['labels']]

        return batch

In [ ]:
# just to make it faster
train_data = df.iloc[0:20]
val_data = df.iloc[20:30]
test_data = df.iloc[30:40]

In [ ]:
train_data = train_data.apply(process_data_to_model_inputs, axis=1)
train_data = train_data.drop(['Textbody', 'Lead'], axis=1)

val_data = val_data.apply(process_data_to_model_inputs, axis=1)
val_data = val_data.drop(['Textbody', 'Lead'], axis=1)

In [ ]:
train_data.head()

,input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,labels
0,"[2, 2155, 7155, 2524, 4046, 4341, 15269, 14826...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 21086, 2018, 6480, 2045, 26204, 14350, 804...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 21086, 2018, 6480, 2045, 26204, 14350, 804..."
1,"[2, 30196, 9688, 11787, 11628, 2005, 11149, 22...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 28589, 6997, 2039, 5833, 6505, 2094, 2005,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 28589, 6997, 2039, 5833, 6505, 2094, 2005,..."
2,"[2, 4405, 2546, 8984, 3576, 2079, 2005, 16667,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 5388, 28306, 7996, 2005, 16667, 3021, 2653...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 5388, 28306, 7996, 2005, 16667, 3021, 2653..."
3,"[2, 22158, 6688, 29536, 2005, 13549, 31735, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 12930, 12887, 6704, 7671, 2837, 20664, 297...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 12930, 12887, 6704, 7671, 2837, 20664, 297..."
4,"[2, 10708, 11447, 2685, 2066, 9763, 31736, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2930, 29814, 2066, 9763, 2033, 2718, 31741...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2930, 29814, 2066, 9763, 2033, 2718, 31741..."


In [ ]:
def to_tensor(x):
  for col in x.keys():
    x[col] = torch.Tensor(x[col])
  return x

train_data = train_data.apply(to_tensor)
val_data = val_data.apply(to_tensor)

## Predict

In [ ]:
def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch['Textbody'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    input_ids = inputs.input_ids.to('cuda')
    attention_mask = inputs.attention_mask.to('cuda')

    outputs = model.generate(input_ids, attention_mask=attention_mask) # the same doesn't work with bert2bert, why??

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch['pred_summary'] = output_str

    return batch

In [ ]:
# example
batch = df.iloc[10]
batch['Textbody']

'Találkozni szeretett volna a Kölcsey Ferenc Gimnázium tanárait menesztő tankerületi vezetővel Soproni Tamás, terézvárosi polgármester. Marosi Betrix azonban nem látta indokoltnak, hogy beszélgessenek. A tankerületi vezető levelét Facebook-oldalán tette közzé a Momentum politikusa. Azt írja Marosi, hogy „természetesen a tankerületi központnak is az az érdeke, hogy az iskolában a helyzet normalizálódjon, a pedagógusok tanítsanak, a diákok tanuljanak”. A nevelés-oktatás feltételeinek biztosítása a VI. kerület állami fenntartású iskoláiban a tankerület feladata és felelőssége,„a tankerületi központ sem kíván beleszólni abba, hogy az Önkormányzat hogyan és kikkel látja el a feladatait.”A tankerületi vezető nem látja indokoltnak, hogy személyesen is találkozzon a kerület polgármesterével, mert a munkáltatói intézkedés „kizárólag a hatályos jogszabályokból fakadó lépés volt”. Marosi szerint az érintett pedagógusok a többszöri figyelmeztetés ellenére is visszatérően megszegték „a pedagógus jo

In [ ]:
inputs = tokenizer(batch['Textbody'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
input_ids = inputs.input_ids.to('cuda')
attention_mask = inputs.attention_mask.to('cuda')
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=50)
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str

['Soproni Tamás szerint a tankerületi központnak is az az érdeke, hogy a tanárok és diákok tanuljanak.']

In [ ]:
batch_size = 64

results = test_data.apply(generate_summary, axis=1)
results = results.drop(['Textbody'], axis=1)
results

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


,Lead,pred_summary
32,Letartóztatták Vietnám leggazdagabb és legtito...,"[A kínai ingatlanmágnás, Truong Kn - ven ( mag..."
33,Futóverseny miatt forgalomkorlátozások lesznek...,[A futóverseny miatt több forgalmas futóesemén...
34,"Egy futópár is feltartóztathatatlan, ha jól tá...",[A sakkrejtvény - sorozat harmadik fordulójába...
35,Orbán Viktor egykori iskolája is szolidaritást...,[A székesfehérvári Teleki Blanka Gimnázium tan...
36,"Perelnek a szülők, miután a magas energiaárakr...",[A tankerület szerint ez nem a településre tar...
37,Verstappen egy századdal nyerte az év legszoro...,[A Mercedes kettős győzelmével zárult a 2022 -...
41,Már tíz százalék feletti régióban vannak egyes...,[A bankok fele már 10 százalék feletti kamatta...
42,"A fő kérdés, hogy arányos válasz-e a tanárok k...",[A Társaság a Szabadságjogokért ( TASZ ) jogvé...
43,Márta István Vidnyánszkyról: Soha nem fogom me...,"[A fesztiválalapításról, a politikai hatalomró..."
44,"A Hatalom Gyűrűi kezd kifutni az időből, de ez...",[A Hatalom Gyűrűi múlt heti eseményei után a v...


## Train further

In [ ]:
# already default
# model.config.decoder_start_token_id == tokenizer.cls_token_id
# model.config.eos_token_id == tokenizer.sep_token_id
# model.config.pad_token_id == tokenizer.pad_token_id
# model.config.vocab_size == bert2bert.config.encoder.vocab_size

In [ ]:
# még nem akarom állítgatni
# bert2bert.config.max_length = 142
# bert2bert.config.min_length = 56
# bert2bert.config.no_repeat_ngram_size = 3
# bert2bert.config.early_stopping = True
# bert2bert.config.length_penalty = 2.0
# bert2bert.config.num_beams = 4

In [ ]:
%%capture
!rm seq2seq_trainer.py
!rm seq2seq_training_args.py
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_training_args.py

In [ ]:
%%capture
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu

In [ ]:
from seq2seq_trainer import Seq2SeqTrainer
from seq2seq_training_args import Seq2SeqTrainingArguments

In [ ]:
batch_size = 64

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=10,
    eval_steps=4
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)

In [ ]:
!pip install datasets
import datasets
rouge = datasets.load_metric("rouge")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 249636609


KeyError: ignored